# 6. Feature Engineering

In [1]:
import pickle
from collections import namedtuple
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pandas as pd
import sys
sys.path.append("/home/angrypark/korean-text-matching-tf/")

from utils.utils import JamoProcessor
from text.tokenizers import SentencePieceTokenizer

Config = namedtuple("config", ["sent_piece_model"])
config = Config("/media/scatter/scatterdisk/tokenizer/sent_piece.100K.model")
processor = JamoProcessor()
tokenizer = SentencePieceTokenizer(config)

def my_word_tokenizer(raw, pos=["Noun", "Alpha", "Verb", "Number"], stopword=[]):
    return [word for word in tokenizer.tokenize(raw)]

def my_char_tokenizer(raw, pos=["Noun", "Alpha", "Verb", "Number"], stopword=[]):
    return [processor.word_to_jamo(word) for word in tokenizer.tokenize(raw)]

In [2]:
from feature_extractor import FeatureExtractor

In [3]:
feature_extractor = FeatureExtractor()

Pre-trained embedding loaded. Number of OOV : 5272 / 90000


/home/angrypark/angryenv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /media/scatter/scatterdisk/reply_matching_model/runs/delstm_1024_nsrandom4_lr1e-3/best_loss/best_loss.ckpt


In [4]:
with open("/home/angrypark/paraphrase_detection/data/test_queries.txt", "r") as f:
    _, queries = zip(*[line.strip().split("\t") for line in f])

with open("/home/angrypark/paraphrase_detection/data/test_replies.txt", "r") as f:
    reply_to_idx_dict = dict()
    replies = list()
    for line in f:
        if not line:
            continue
        splits = line.strip().split("\t")
        reply_to_idx_dict[splits[1]] = splits[0]
        replies.append(splits[1])

In [5]:
submission_data = {}
length = len(replies)

In [6]:
from tqdm import tqdm

In [7]:
queries[23:30]

('너 도대체 정체가 뭔데?',
 '넌 어떻게 만들어졌니?',
 '나 결혼 할 수 있을까요?',
 '이제 점심먹을거야',
 '오늘도 나 꿈꿨다',
 '오늘 먹은 고기 진짜 맛있었음',
 '비 엄청온다 ㄷㄷ')

In [67]:
pd.read_csv("../data/my_data/train_set.csv").head()

,ab_probs,ba_probs,edit_distance,semantic_sim,substring_ratio,tfidf_char_sim,tfidf_word_sim
0,0.860981,0.790133,0.565217,0.690985,0.173913,0.343242,0.119655
1,0.935994,0.974954,0.428571,0.781252,0.357143,0.754234,0.422782
2,0.998473,0.993880,0.680000,0.704285,0.240000,0.322436,0.000000
3,0.515418,0.149820,0.847826,0.557047,0.043478,0.032540,0.000000
4,0.999465,0.999118,0.250000,0.887023,0.583333,0.683980,0.252547


In [8]:
total = pd.DataFrame()
for query in tqdm(queries):
    A, B = [query]*length, replies
    extracted = feature_extractor.extract_features(A, B)
    extracted = pd.DataFrame(extracted)
    extracted["sentence_A"] = query
    extracted["sentence_B"] = pd.Series(replies)
    total = pd.concat([total, extracted])

100%|██████████| 600/600 [06:06<00:00,  1.64it/s]


In [50]:
A, B = ["사랑해요"] * length, replies

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [60]:
word_sim = list()
for a, b in zip(A, B):
    word_sim.append(cosine_similarity(feature_extractor.tfidf_word_vectorizer.transform([a]), 
                                      feature_extractor.tfidf_word_vectorizer.transform([b])))

In [58]:
[(a, b) for a, b in zip(A, B)][0]

('사랑해요', '흑흑 일 많은데 졸리다')

In [59]:
feature_extractor.tfidf_word_vectorizer.transform(['사랑해요'])

<1x91743 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [51]:
extracted = feature_extractor.extract_features(A, B)

In [53]:
extracted["tfidf_word_sim"]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [116]:
def model_to_submission(model, name):
    def load_data():
        base_dir = "/home/angrypark/paraphrase_detection/data/my_data/"
        train_set = pd.read_csv(base_dir + "train_set.csv")
        val_set = pd.read_csv(base_dir + "val_set.csv")
        test_set = pd.read_csv(base_dir + "test_set.csv")
        with open(base_dir + "train.txt", "r") as f:
            _, _, train_labels = zip(*[line.strip().split("\t") for line in f if line])
            train_labels = [1 if l=="1" else 0 for l in train_labels]
        with open(base_dir + "val.txt", "r") as f:
            _, _, val_labels = zip(*[line.strip().split("\t") for line in f if line]) 
            val_labels = [1 if l=="1" else 0 for l in val_labels]
        with open(base_dir + "test.txt", "r") as f:
            _, _, test_labels = zip(*[line.strip().split("\t") for line in f if line])
            test_labels = [1 if l=="1" else 0 for l in test_labels]
            
        return train_set, val_set, test_set, train_labels, val_labels, test_labels
    
    with open("/home/angrypark/paraphrase_detection/data/test_queries.txt", "r") as f:
        _, queries = zip(*[line.strip().split("\t") for line in f])
        
    with open("/home/angrypark/paraphrase_detection/data/test_replies.txt", "r") as f:
        reply_to_idx_dict = dict()
        replies = list()
        for line in f:
            splits = line.strip().split("\t")
            reply_to_idx_dict[splits[1]] = splits[0]
            replies.append(splits[1])
    
    train_set, val_set, test_set, train_labels, val_labels, test_labels = load_data()
    print("Train accuracy : {:.5f}".format(model.score(train_set, train_labels)))
    print("Val   accuracy : {:.5f}".format(model.score(val_set, val_labels)))
    print("Test  accuracy : {:.5f}".format(model.score(test_set, test_labels)))
    
    length = len(replies)
    submit_set = pd.read_csv("../data/submit_set.csv")
    features = list(submit_set.columns).copy()
    features.remove("sentence_A")
    features.remove("sentence_B")
    
    predictions = list()
    for i, query in enumerate(queries):
        data = submit_set.iloc[i*length:(i+1)*length]
        probs = [p[1] for p in model.predict_proba(data[features]).tolist()]
        scores = [(reply, score) for reply, score in zip(replies, probs)]
        predict = sorted(scores, key=lambda x: x[1], reverse=True)[0][0]
        predictions.append(reply_to_idx_dict[predict])
    submission = pd.read_csv("../submission/sample_submission.csv")
    submission["id_script"] = pd.Series(predictions)
    submission.to_csv("../submission/{}.csv".format(name), index=False)
    return submission

In [117]:
xgb_new = pickle.load(open("../models/new_xgb.pkl", "rb"))

In [118]:
_ = model_to_submission(xgb_new, name="xgb_new")

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.88728
Val   accuracy : 0.73200
Test  accuracy : 0.76550


/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [98]:
xgb_old = pickle.load(open("../models/xgb.pkl", "rb"))

In [99]:
xgb_submission = model_to_submission(xgb_old, name="xgb_old")

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.88795
Val   accuracy : 0.74307
Test  accuracy : 0.77395


/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [100]:
xgb_submission

,id_query,id_script
0,0,152
1,1,29
2,2,26
3,3,168
4,4,40
5,5,31
6,6,164
7,7,149
8,8,67
9,9,24


In [ ]:
def make_submission(probs, name):
    """probs를 집어넣으면 바로 submission 파일 생성
    :param probs: {query: [(reply, score), (reply, score)...]} 형태
    :param name: submission의 이름
    """
    with open("/home/angrypark/paraphrase_detection/data/test_replies.txt", "r") as f:
        reply_to_idx_dict = {reply: idx for idx, reply in [line.strip().split("\t") for line in f]}
    